<a href="https://colab.research.google.com/github/tilly963963/pytorch_nlp/blob/main/yelp_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/Shareddrives/python教學/PyTorchNLPBook-master/chapters/chapter_3/")

https://blog.csdn.net/weixin_42028364/article/details/81675021

https://allenlu2007.wordpress.com/2019/02/07/%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92%E7%9A%84-pytorch-load-dataset/

In [ ]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [ ]:
def add_token(token, token_to_idx=None, idx_to_token=None):
  if token in token_to_idx:#當查詢用的嗎?
    index = token_to_idx[token]
    print("token in token_to_idx=",token )
  else:
    index = len(token_to_idx)#目前長度
    token_to_idx[token] = index
    idx_to_token[index] = token
  # print("token(word)=",token,"index=",index)#>25次出現 依照順序 不重複的的
  # print("token_to_idx=",token_to_idx)
  # print("idx_to_token=",idx_to_token)

  return index, token_to_idx, idx_to_token   

In [ ]:
def Vocabulary__init__(add_unk=True, unk_token="<UNK>"):
  # if token_to_idx is None:
  token_to_idx = {}
  # token_to_idx = token_to_idx
  # idx_to_token = {idx: token 
  #             for token, idx in token_to_idx.items()}
  # # print("idx_to_token=",idx_to_token) {}
  # add_unk = add_unk
  # unk_token = unk_token

  idx_to_token={}
  unk_index = -1

  if add_unk:#若有沒見過的字的處理
    unk_index, token_to_idx, idx_to_token = add_token(unk_token, token_to_idx, idx_to_token) 

  print("unk_index",unk_index)
  return unk_index, token_to_idx, idx_to_token 

In [ ]:
def from_dataframe(review_df, cutoff=25):
  '''
  沒見過的字的處理
  '''
  review_vocab, review_token_to_idx, review_idx_to_token  = Vocabulary__init__(add_unk=True)
  print("review_vocab, review_token_to_idx, review_idx_to_token=",review_vocab, review_token_to_idx, review_idx_to_token)
  # review_vocab, review_token_to_idx, review_idx_to_token= 0 {'<UNK>': 0} {0: '<UNK>'}
  rating_vocab, rating_token_to_idx, rating_idx_to_token  = Vocabulary__init__(add_unk=False)
  print("rating_vocab, rating_token_to_idx, rating_idx_to_token=",rating_vocab, rating_token_to_idx, rating_idx_to_token)
  # rating_vocab, rating_token_to_idx, rating_idx_to_token= -1 {} {}
  '''
  真實出現字的處理
  '''
  # Add ratings
  for rating in sorted(set(review_df.rating)):#['negative', 'positive']
    rating_vocab, rating_token_to_idx, rating_idx_to_token = add_token(rating, rating_token_to_idx, rating_idx_to_token)
  print("rating_vocab, rating_token_to_idx, rating_idx_to_token=",rating_vocab, rating_token_to_idx, rating_idx_to_token)
  # rating_vocab, rating_token_to_idx, rating_idx_to_token= 1 {'negative': 0, 'positive': 1} {0: 'negative', 1: 'positive'}
  word_counts = Counter()
  for review in review_df.review:
    for word in review.split(" "):
      if word not in string.punctuation:
        word_counts[word] += 1 #累積出現次數

  # print("word_counts.items()",word_counts.items())#每個字出現次數
  # word_counts.items() dict_items([('overpriced', 53), 
  for word, count in word_counts.items():
    if count >150:#cutoff:
        review_vocab, review_token_to_idx, review_idx_to_token = add_token(word, review_token_to_idx, review_idx_to_token)#依照順序給編號
  print("review_vocab=",review_vocab)#常用的字的數量
  review_vocab = review_vocab+1
  rating_vocab = rating_vocab+1
  return review_vocab, review_token_to_idx, review_idx_to_token ,rating_vocab, rating_token_to_idx, rating_idx_to_token 

In [ ]:
review_csv='data/yelp/reviews_with_splits_lite.csv'
review_df = pd.read_csv(review_csv)
train_review_df = review_df[review_df.split=='train']
print(review_df.rating)
print("--")
print(sorted(set(review_df.rating)))
review_vocab, review_token_to_idx, review_idx_to_token ,rating_vocab, rating_token_to_idx, rating_idx_to_token = from_dataframe(train_review_df)

0       negative
1       negative
2       negative
3       negative
4       negative
          ...   
3795    positive
3796    positive
3797    positive
3798    positive
3799    positive
Name: rating, Length: 3800, dtype: object
--
['negative', 'positive']
unk_index 0
review_vocab, review_token_to_idx, review_idx_to_token= 0 {'<UNK>': 0} {0: '<UNK>'}
unk_index 0
rating_vocab, rating_token_to_idx, rating_idx_to_token= 0 {} {}
rating_vocab, rating_token_to_idx, rating_idx_to_token= 1 {'negative': 0, 'positive': 1} {0: 'negative', 1: 'positive'}
review_vocab= 306


In [ ]:
class ReviewClassifier(nn.Module):
    """ a simple perceptron based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(ReviewClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, 
                             out_features=1)
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,)
        """
        y_out = self.fc1(x_in).squeeze()
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

In [ ]:
class ReviewDataset(Dataset):
  def __init__(self, review_df, vectorizer, review_vocab, review_token_to_idx, review_idx_to_token ,rating_vocab, rating_token_to_idx, rating_idx_to_token ):
      print("===ReviewDataset__init__===")
      self.review_df = review_df
      self._vectorizer = vectorizer#307

      self.train_df = self.review_df[self.review_df.split=='train']
      self.train_size = len(self.train_df)

      self.val_df = self.review_df[self.review_df.split=='val']
      self.validation_size = len(self.val_df)

      self.test_df = self.review_df[self.review_df.split=='test']
      self.test_size = len(self.test_df)

      self._lookup_dict = {'train': (self.train_df, self.train_size),
                            'val': (self.val_df, self.validation_size),
                            'test': (self.test_df, self.test_size)}
      self.review_vocab, self.review_token_to_idx, self.review_idx_to_token ,self.rating_vocab, self.rating_token_to_idx, self.rating_idx_to_token = review_vocab, review_token_to_idx, review_idx_to_token ,rating_vocab, rating_token_to_idx, rating_idx_to_token
      # self.set_split('train')
      split="train"
      target_split = split#??
      self._target_df, self._target_size = self._lookup_dict[split]
      print("target_df")
      print(self._target_df)
      print("target_size=",self._target_size)    
  def set_split(self, split="train"):
      self._target_split = split
      self._target_df, self._target_size = self._lookup_dict[split]
      print("self._target_df")
      print(self._target_df)
      print("self._target_size=",self._target_size)
  def review_lookup_token(self, token):
      return self.review_token_to_idx.get(token, 0)#返回位置
  def rating_lookup_token(self, token):
      return self.rating_token_to_idx[token]##返回位置
  def rating_lookup_index(self, index):
    return self.rating_idx_to_token[index]

  def vectorize(self, review, review_vocab):
      print("review=",review)
      one_hot = np.zeros(review_vocab, dtype=np.float32)
      # print("one_hot.size",one_hot.shape)#306
      for token in review.split(" "):
        if token not in string.punctuation:#從字去找位置idx
            one_hot[self.review_lookup_token(token)] = 1#在一句話中無法考慮相同的字出現次數 只會在該位置=1只會在該位置=1
      # print("one_hot.shape=",one_hot.shape)
      return one_hot
  def __len__(self):
      return self._target_size
  def __getitem__(self, index):
      """the primary entry point method for PyTorch datasets
      
      Args:
          index (int): the index to the data point 
      Returns:
          a dictionary holding the data point's features (x_data) and label (y_target)
      """
      print("ReviewDataset __getitem__()")
      row = self._target_df.iloc[index]
      # print("row=",row)#一句話
      review_vector = self.vectorize(row.review, self._vectorizer)
      # print("row.review=",row.review)
      # print("np.array(review_vector).shape=",np.array(review_vector).shape)
      rating_index = self.rating_lookup_token(row.rating)
      # print("np.array(rating_index).shape=",rating_index)
      # np.array(review_vector).shape= (3879,)
      # np.array(rating_index).shape= ()
      print("?")
      return {'x_data': review_vector,
              'y_target': rating_index}

def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):

    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=False, drop_last=drop_last)#Dataset_obj
    print("generate_batches()")
    # print("dataloader",dataloader) 
    # x_data= dataloader.__iter__().__next__() #torch.utils.data.dataloader.DataLoader
    # print("x_data=",x_data) 
    for data_dict in dataloader: # x_data = dataloader.__iter__().__next__()
        '''
        當迭代dataloader時會自動呼叫 
        class DataLoader(object)  
          def __iter__(self): 
            return DataLoaderIter(self)
        #class DataLoaderIter(object): 
          def __next__(self):
            batch = self.collate_fn([self.dataset[i] for i in indices])
            self.dataset[i] => __getitem__() 每一次回傳批次個  用self.collate_fn 打包成批次個
        '''
        out_data_dict = {}
        print("data_dict.items()")
        '''
        dict_items([('x_data', tensor([[1.,...],
        [1.,.]])), ('y_target', tensor([0, 0]))])
        '''
        print(data_dict.items())
        for name, tensor in data_dict.items():#__getitem__
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [ ]:
dataset = ReviewDataset(review_df, review_vocab, review_vocab, review_token_to_idx, review_idx_to_token ,rating_vocab, rating_token_to_idx, rating_idx_to_token)

===ReviewDataset__init__===
target_df
        rating                                             review  split
0     negative  overpriced food . they were out of a lot of co...  train
1     negative  we had been hearing rave reviews about this pi...  train
2     negative  needing a food fix while walking through vario...  train
3     negative  walmart makes me cringe . not only for all kin...  train
4     negative  we love this place , granted our first time wa...  train
...        ...                                                ...    ...
3225  positive  wow the pasta is really good at this place . t...  train
3226  positive  they re a great local alternative to starbucks...  train
3227  positive  i wish there was a way i could give this place...  train
3228  positive  went there yesterday and found that the place ...  train
3229  positive  all in all , this is one of my fave spots to g...  train

[2660 rows x 3 columns]
target_size= 2660


In [ ]:
classifier = ReviewClassifier(num_features=review_vocab)#重複>25次的字有多少個 一句話有3879個數字(0or1 全部不重複大於次數25次的字有3879個)

In [ ]:
# # Check CUDA
# args = Namespace(
#     # Data and Path information
#     frequency_cutoff=25,
#     model_state_file='model.pth',
#     review_csv='data/yelp/reviews_with_splits_lite.csv',
#     # review_csv='data/yelp/reviews_with_splits_full.csv',
#     save_dir='model_storage/ch3/yelp/',
#     vectorizer_file='vectorizer.json',
#     # No Model hyper parameters
#     # Training hyper parameters
#     batch_size=2,
#     early_stopping_criteria=5,
#     learning_rate=0.001,
#     num_epochs=100,
#     seed=1337,
#     # Runtime options
#     catch_keyboard_interrupt=True,
#     cuda=True,
#     expand_filepaths_to_save_dir=True,
#     reload_from_files=False,
# )
# args.cuda = True
# if not torch.cuda.is_available():
#     args.cuda = False

# print("Using CUDA: {}".format(args.cuda))

# args.device = torch.device("cuda" if args.cuda else "cpu")
# classifier = classifier.to(args.device)

# loss_func = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,mode='min', factor=0.5, patience=1)
# for epoch_index in range(1):              
#   batch_generator = generate_batches(train_data,batch_size=args.batch_size,device=args.device) 
#   # batch_generator.__iter__().__next__()
#   # print(next(batch_generator))
#   print("batch_generator=",enumerate(batch_generator)) 
#   # for batch_index, batch_dict in enumerate(batch_generator):
#   #   print(batch_index)

In [ ]:
# Check CUDA
args = Namespace(
    # Data and Path information
    frequency_cutoff=25,
    model_state_file='model.pth',
    review_csv='data/yelp/reviews_with_splits_lite.csv',
    # review_csv='data/yelp/reviews_with_splits_full.csv',
    save_dir='model_storage/ch3/yelp/',
    vectorizer_file='vectorizer.json',
    # No Model hyper parameters
    # Training hyper parameters
    batch_size=2,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=100,
    seed=1337,
    # Runtime options
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)
args.cuda = True
if not torch.cuda.is_available():
    args.cuda = False

print("Using CUDA: {}".format(args.cuda))

args.device = torch.device("cuda" if args.cuda else "cpu")
classifier = classifier.to(args.device)

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,mode='min', factor=0.5, patience=1)
for epoch_index in range(3):              
  dataset.set_split('train')
  batch_generator = generate_batches(dataset,batch_size=args.batch_size,device=args.device) #yiekd => print(next(batch_generator))

  running_loss = 0.0
  running_acc = 0.0                               
  for batch_index, batch_dict in enumerate(batch_generator):
      optimizer.zero_grad()
      # print("batch_index",batch_index)
      y_pred = classifier(x_in=batch_dict['x_data'].float())
      # print("y_pred=",np.array(y_pred).shape)

      # step 3. compute the loss
      loss = loss_func(y_pred, batch_dict['y_target'].float())
      loss_t = loss.item()
      running_loss += (loss_t - running_loss) / (batch_index + 1)

      # step 4. use loss to produce gradients
      loss.backward()

      # step 5. use optimizer to take gradient step
      optimizer.step()
      # -----------------------------------------
      # compute the accuracy
      # acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
      # running_acc += (acc_t - running_acc) / (batch_index + 1)


In [ ]:
dataset.set_split('val')
batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):

    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())

    # step 3. compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    # acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    # running_acc += (acc_t - running_acc) / (batch_index + 1)
    

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [ ]:
  # def vectorize(self, review, review_vocab):
  #     print("review=",review)
  #     one_hot = np.zeros(review_vocab, dtype=np.float32)
  #     # print("one_hot.size",one_hot.shape)#306
  #     for token in review.split(" "):
  #       if token not in string.punctuation:#從字去找位置idx
  #           one_hot[self.review_lookup_token(token)] = 1#在一句話中無法考慮相同的字出現次數 只會在該位置=1只會在該位置=1
  #     # print("one_hot.shape=",one_hot.shape)
  #     return one_hot

In [ ]:
def predict_rating(review, classifier, dataset, decision_threshold=0.5):
    """Predict the rating of a review
    
    Args:
        review (str): the text of the review
        classifier (ReviewClassifier): the trained model
        vectorizer (ReviewVectorizer): the corresponding vectorizer
        decision_threshold (float): The numerical boundary which separates the rating classes
    """
    review = preprocess_text(review)
    
    vectorized_review = torch.tensor(dataset.vectorize(review, dataset.review_vocab))#one hot
    print("vectorized_review.view(1, -1)=",vectorized_review.view(1, -1))
    result = classifier(vectorized_review.view(1, -1))
    print("result=",result)
    
    probability_value = F.sigmoid(result).item()
    print("probability_value=",probability_value)
    index = 1
    if probability_value < decision_threshold:
        index = 0
    print("index=",index)
    print(dataset.rating_lookup_index(index))
    return dataset.rating_lookup_index(index)

In [ ]:
test_review = "this is a pretty awesome book"
dataset.set_split('train')
classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, dataset, decision_threshold=0.5)
print("{} -> {}".format(test_review, prediction))

self._target_df
        rating                                             review  split
0     negative  overpriced food . they were out of a lot of co...  train
1     negative  we had been hearing rave reviews about this pi...  train
2     negative  needing a food fix while walking through vario...  train
3     negative  walmart makes me cringe . not only for all kin...  train
4     negative  we love this place , granted our first time wa...  train
...        ...                                                ...    ...
3225  positive  wow the pasta is really good at this place . t...  train
3226  positive  they re a great local alternative to starbucks...  train
3227  positive  i wish there was a way i could give this place...  train
3228  positive  went there yesterday and found that the place ...  train
3229  positive  all in all , this is one of my fave spots to g...  train

[2660 rows x 3 columns]
self._target_size= 2660
review= this is a pretty awesome book
vectorized_review.vie

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
  vectorizer_filepath='model_storage/ch3/yelp/'+'vectorizer.json'
  with open(vectorizer_filepath, "w") as fp:
    json.dump(self._vectorizer.to_serializable(), fp)

NameError: ignored